In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("dataset_Similarity_calculation.csv")

train_rows, val_rows, test_rows = [], [], []

for dataset_name, subset in df.groupby("dataset"):
    # 1. Todos los códigos únicos del dataset
    codes = pd.unique(subset[["idcode1", "idcode2"]].values.ravel())

    # 2. Split sobre códigos, no sobre filas
    codes_train, codes_temp = train_test_split(
        codes, test_size=0.2, random_state=42, shuffle=True
    )
    codes_val, codes_test = train_test_split(
        codes_temp, test_size=0.5, random_state=42, shuffle=True
    )

    # 3. Máscaras: sólo filas cuyos dos códigos vivan en el mismo split
    train_mask = subset["idcode1"].isin(codes_train) & subset["idcode2"].isin(codes_train)
    val_mask   = subset["idcode1"].isin(codes_val)   & subset["idcode2"].isin(codes_val)
    test_mask  = subset["idcode1"].isin(codes_test)  & subset["idcode2"].isin(codes_test)

    train_rows.append(subset[train_mask])
    val_rows.append(subset[val_mask])
    test_rows.append(subset[test_mask])

    print(f"{dataset_name}: {train_mask.sum()} train, {val_mask.sum()} val, {test_mask.sum()} test")

# 4. Concatenamos y comprobamos que no hay contaminación
train_df = pd.concat(train_rows).reset_index(drop=True)
val_df   = pd.concat(val_rows).reset_index(drop=True)
test_df  = pd.concat(test_rows).reset_index(drop=True)

train_codes = set(train_df["idcode1"]).union(train_df["idcode2"])
val_codes   = set(val_df["idcode1"]).union(val_df["idcode2"])
test_codes  = set(test_df["idcode1"]).union(test_df["idcode2"])

assert train_codes.isdisjoint(val_codes)
assert train_codes.isdisjoint(test_codes)
assert val_codes.isdisjoint(test_codes)

# 5. Guardamos
train_df.to_csv("train.csv", index=False, quoting=1)
val_df.to_csv("val.csv", index=False, quoting=1)
test_df.to_csv("test.csv", index=False, quoting=1)


KeyError: "None of [Index(['file_a', 'file_b'], dtype='object')] are in the [columns]"